### Test of CV functionality. 

Most storage related tests are nosetests now. So look in `testcollectivevariable.py` please.

In [1]:
import openpathsampling as paths
import numpy as np
import openpathsampling.engines.openmm as peng

Load first frame from test pdb.

In [2]:
template = peng.snapshot_from_pdb('../resources/AD_initial_frame.pdb')

Create a simple CV without an underlying function.

In [3]:
cv0 = paths.CollectiveVariable('func0')

Create a complicated function.

In [4]:
center = 1
def dist(snapshot, center, np):
    return np.sum(snapshot.coordinates._value[0]) - center

Create collective variable from this function. Note that you have to specify `center` and `np` to make this work

In [5]:
cv1 = paths.FunctionCV('func1', dist, center=center, np=np, cv_time_reversible=False).with_diskcache()
cv2 = paths.FunctionCV('func2', dist, center=center, np=np, cv_wrap_numpy_array=True).with_diskcache()
cv3 = paths.FunctionCV('func3', dist, center=center, np=np, cv_wrap_numpy_array=True, cv_time_reversible=True).with_diskcache()

Create storage to test save and load.

In [6]:
#! lazy
storage = paths.Storage('can_be_deleted.nc', mode='w')
print(storage.snapshots.save(template))

47581121712829014922446120121750519820


Save CV

In [7]:
#! lazy
print(storage.save([cv0, cv1, cv2, cv3]))

[(store.cvs[Attribute] : 4 object(s), 4, 47581121712829014922446120121750519822L), (store.cvs[Attribute] : 4 object(s), 4, 47581121712829014922446120121750519824L), (store.cvs[Attribute] : 4 object(s), 4, 47581121712829014922446120121750519826L), (store.cvs[Attribute] : 4 object(s), 4, 47581121712829014922446120121750519828L)]


In [8]:
storage.cvs.index

{47581121712829014922446120121750519822L: 0,
 47581121712829014922446120121750519824L: 1,
 47581121712829014922446120121750519826L: 2,
 47581121712829014922446120121750519828L: 3}

Set the CV value for the storage.template

In [9]:
cv0[template] = 10.0

and create a storage for it

In [10]:
storage.cvs.add_diskcache(cv0, allow_incomplete=True)

And we should have a float store

In [11]:
storage.cvs.sync(cv0)

In [12]:
assert(cv0._store_dict.value_store.vars['value'][:] == [10.0])

Test function for reversed template.

In [13]:
dd = dist(template.reversed, center, np)
print(dd)

-1.08320000023


Create another CV. This time using the `from_template` function.

In [14]:
print(cv0([template, template]))
print(cv1([template, template]))

[10.0, 10.0]
[-1.0832000002264977, -1.0832000002264977]


In [15]:
print(type(cv0([template, template])))
print(type(cv0([template, template])[0]))
print(type(cv1([template, template])))
print(type(cv1([template, template])[0]))

<class 'list'>
<class 'float'>
<class 'list'>
<class 'numpy.float64'>


In [16]:
#! skip
print(storage.cvs.variables['json'][:])

[ '{"_cls":"CollectiveVariable","_dict":{"name":"func0","cv_time_reversible":false}}'
 '{"_cls":"FunctionCV","_dict":{"name":"func1","cv_time_reversible":false,"f":{"_marshal":"4wMAAAAAAAAAAwAAAAMAAABDAAAAcxYAAAB8AmoAfABqAWoCZAEZAIMBfAEYAFMAKQJO6QAAAAApA9oDc3Vt2gtjb29yZGluYXRlc9oGX3ZhbHVlKQPaCHNuYXBzaG902gZjZW50ZXLaAm5wqQByCAAAAPoePGlweXRob24taW5wdXQtNC03MzIzOGI2Njk5ODc+2gRkaXN0AgAAAHMCAAAAAAE=","_global_vars":[],"_module_vars":[]},"cv_requires_lists":false,"cv_wrap_numpy_array":false,"cv_scalarize_numpy_singletons":false,"kwargs":{"center":1,"np":{"_import":"numpy"}}}}'
 '{"_cls":"FunctionCV","_dict":{"name":"func2","cv_time_reversible":false,"f":{"_marshal":"4wMAAAAAAAAAAwAAAAMAAABDAAAAcxYAAAB8AmoAfABqAWoCZAEZAIMBfAEYAFMAKQJO6QAAAAApA9oDc3Vt2gtjb29yZGluYXRlc9oGX3ZhbHVlKQPaCHNuYXBzaG902gZjZW50ZXLaAm5wqQByCAAAAPoePGlweXRob24taW5wdXQtNC03MzIzOGI2Njk5ODc+2gRkaXN0AgAAAHMCAAAAAAE=","_global_vars":[],"_module_vars":[]},"cv_requires_lists":false,"cv_wrap_numpy_array":true,"cv_scalarize_numpy

In [17]:
cv0j = storage.cvs.vars['json'][0]
cv1j = storage.cvs.vars['json'][1]
cv2j = storage.cvs.vars['json'][2]
cv3j = storage.cvs.vars['json'][3]

In [18]:
res = cv0j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv1j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv2j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv3j([template, template, template])
assert(res[0] == res[1] == res[2])

In [19]:
t = paths.Trajectory([template] * 4)

In [20]:
storage.save(t)

(store.trajectories[Trajectory] : 1 object(s),
 1,
 47581121712829014922446120121750519898L)

In [21]:
def ff(t, cv3):
    return max(cv3(t))

In [22]:
a = paths.netcdfplus.FunctionAttribute('max_cv3', paths.Trajectory, ff, cv3=cv3)

In [23]:
storage.attributes.save(a)

47581121712829014922446120121750519900L

In [24]:
storage.attributes.index[a.__uuid__]

0

In [25]:
a(t)

-1.0832000002264977

In [26]:
p = paths.netcdfplus.LoaderProxy(storage.trajectories, t.__uuid__)

In [29]:
p._load_()

Trajectory[4]

In [30]:
a._eval(p)

-1.0832000002264977

In [31]:
a(paths.netcdfplus.LoaderProxy(storage.trajectories, t.__uuid__))

-1.0832000002264977

In [32]:
storage.trajectories.add_attribute(paths.netcdfplus.ValueStore, a, t)

store.trajectories_max_cv3[None/ANY] : (not created)

In [29]:
storage.trajectories.index

{183378025328531209523840242851002712154L: 0}

In [27]:
a.__dict__

{'__uuid__': 275237467320013943139557271650716090460L,
 '_cache_dict': <openpathsampling.netcdfplus.chaindict.CacheChainDict at 0x11a04db90>,
 '_eval_dict': <openpathsampling.netcdfplus.chaindict.Function at 0x11a04d110>,
 '_iterables': (list, tuple),
 '_name': 'max_cv3',
 '_name_fixed': True,
 '_post': <openpathsampling.netcdfplus.chaindict.ExpandSingle at 0x11a2fecd0>,
 '_single_dict': <openpathsampling.netcdfplus.chaindict.ExpandSingle at 0x11a2fecd0>,
 '_singles': (),
 '_store_dict': None,
 'cv_callable': <function __main__.ff>,
 'cv_requires_lists': False,
 'cv_scalarize_numpy_singletons': False,
 'cv_wrap_numpy_array': False,
 'diskcache_allow_incomplete': False,
 'diskcache_chunksize': 256,
 'diskcache_enabled': False,
 'diskcache_template': None,
 'key_class': openpathsampling.engines.trajectory.Trajectory,
 'kwargs': {'cv3': <openpathsampling.collectivevariable.FunctionCV at 0x11a090410>},
 'stores': []}

In [28]:
storage.snapshots.index

{183378025328531209523840242851002712076L: 0L}

In [28]:
storage.attributes.name_idx

{'max_cv3': {0}}

In [32]:
print a._cache_dict.cache.items()

[(Trajectory[4], -1.0832000002264977)]


In [34]:
storage.attributes.set_cache_store(a)

In [35]:
a._store_dict

In [36]:
storage.attributes.has_cache(a)

True

In [38]:
storage.attributes.sync(a)

AttributeError: 'TrajectoryStore' object has no attribute 'sync_cv'